In [2]:
from logging import root
import os
from os import listdir
from os.path import isfile, join
import re
import openbabel
from openbabel import pybel
from debugpy import connect
from soupsieve import select
from zmq import CONFLATE
import rdkit
import pytz
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem import AllChem, MolStandardize, CombineMols
from rdkit.Chem.Scaffolds import rdScaffoldNetwork
import copy
import pandas as pd
import time
from datetime import datetime
import psycopg2
from google.cloud import storage
from Bio.PDB import PDBParser, PDBIO, Select
import paramiko
import getpass
from scp import SCPClient
import tarfile
from sqlalchemy import desc, select
import sqlalchemy
from rdkit.Chem import PandasTools

In [14]:
path = '/Users/song-inhyeok/Documents/BIChem/1V4S/gen_mol'
path2 = '/Users/song-inhyeok/Documents/BIChem/1V4S/liGAN_input.types'
    
file_list = [ 'DEMO_' + str(i) + '_lig_gen_fit_add_uff.sdf' for i in range(23)]
sample=file_list[0]
#print(file_list)

list_file = open(path2, 'r')
lines = list_file.readlines()

df = pd.DataFrame()

for i, file in enumerate(file_list):
            #if i == 0 or i == 1:
            #    continue
            file_name = path + '/' + file
            sdf = Chem.SDMolSupplier(file_name, sanitize = False)
            mol_3d_list = open(file_name, 'r').read().split('$$$$')
            
            conf_id = lines[i].split('/')[-1].split('_')[-1].split('.')[0].replace('\n', '')
            
            for j, mol in enumerate(sdf):
                mol_3d = mol_3d_list[j].lstrip()
                df = df.append({{'id': i, 'mol_3d': mol_3d}, ignore_index= })
                if mol == None:
                    f = open('error_list_t.sdf', 'a')
                    f.write(mol_3d)
                    f.write('\n')
                    f.close()
                    g  = open('erro_mol_t.txt', 'a')
                    g.write(file_name + str(j) +'번째 화합물 오류'+'\n')
                    g.close()
                    continue


[H]OC12c3c([H])c([H])c4nc(=O)nc4c3C1([H])C([H])=C([H])OC([H])([H])C2([H])[H]
[H]OC([H])(c1c([H])c([H])c2nc(=O)nc2c1[H])C1([H])C([H])([H])C1(O[H])C([H])([H])[H]
[H]OC1(c2c([H])c([H])c3nc(=O)nc3c2[H])C2([H])OC([H])=C([H])C21[H]
[H]N=C(c1c([H])c([H])c2c1nc(=O)n2[H])[C@@]1([H])C([H])([H])[C@]2([H])O[C@@]21[H]
[H]Oc(nc1c([H])c([H])c([H])c(OC([H])([H])C([H])([H])C([H])([H])OC([H])([H])[H])c1[H])C([H])([H])[H]
[H]c1c(C(=O)C2([H])C([H])([H])C([H])([H])C2([H])[H])c([H])c2nc(=O)nc2c1[H]
[H]OC([H])([H])c1nc2c([H])c(C3(O[H])C4([H])C(=O)C([H])([H])C43[H])c([H])c([H])c12
[H]c1nc2c3c4(c([H])c12)OC34C([H])([H])C([H])([H])[H]
[H]OC([H])(C1([H])C([H])([H])C([H])([H])C1([H])[H])c1([H])c([H])c([H])c2nc([H])nc2c1[H]
[H]C1=C([H])C(C([H])(c2c([H])c([H])c3c(c2[H])c3([H])C([H])([H])[H])N([H])[H])=C([H])O1
[H]OC1(c2c([H])c([H])c3nc(=O)nc3c2[H])C([H])=C([H])C([H])([H])C([H])([H])C1([H])[H]
[H]Oc1c(C2([H])C([H])=C([H])OC2([H])F)c([H])c2nc(=O)c([H])c2c1[H]
[H]OC([H])=Nc1c([H])c([H])c([H])c([C@@]2(O[H])C([H])=C([H]

In [18]:
file_list

['DEMO_0_lig_gen_fit_add_uff.sdf',
 'DEMO_1_lig_gen_fit_add_uff.sdf',
 'DEMO_2_lig_gen_fit_add_uff.sdf',
 'DEMO_3_lig_gen_fit_add_uff.sdf',
 'DEMO_4_lig_gen_fit_add_uff.sdf',
 'DEMO_5_lig_gen_fit_add_uff.sdf',
 'DEMO_6_lig_gen_fit_add_uff.sdf',
 'DEMO_7_lig_gen_fit_add_uff.sdf',
 'DEMO_8_lig_gen_fit_add_uff.sdf',
 'DEMO_9_lig_gen_fit_add_uff.sdf',
 'DEMO_10_lig_gen_fit_add_uff.sdf',
 'DEMO_11_lig_gen_fit_add_uff.sdf',
 'DEMO_12_lig_gen_fit_add_uff.sdf',
 'DEMO_13_lig_gen_fit_add_uff.sdf',
 'DEMO_14_lig_gen_fit_add_uff.sdf',
 'DEMO_15_lig_gen_fit_add_uff.sdf',
 'DEMO_16_lig_gen_fit_add_uff.sdf',
 'DEMO_17_lig_gen_fit_add_uff.sdf',
 'DEMO_18_lig_gen_fit_add_uff.sdf',
 'DEMO_19_lig_gen_fit_add_uff.sdf',
 'DEMO_20_lig_gen_fit_add_uff.sdf',
 'DEMO_21_lig_gen_fit_add_uff.sdf',
 'DEMO_22_lig_gen_fit_add_uff.sdf']

In [36]:
file_name = []
for file in file_list:
    file_name.append(path+file)

In [37]:
print(file_name)

['/Users/song-inhyeok/Documents/BIChem/1V4S/gen_molDEMO_0_lig_gen_fit_add_uff.sdf', '/Users/song-inhyeok/Documents/BIChem/1V4S/gen_molDEMO_1_lig_gen_fit_add_uff.sdf', '/Users/song-inhyeok/Documents/BIChem/1V4S/gen_molDEMO_2_lig_gen_fit_add_uff.sdf', '/Users/song-inhyeok/Documents/BIChem/1V4S/gen_molDEMO_3_lig_gen_fit_add_uff.sdf', '/Users/song-inhyeok/Documents/BIChem/1V4S/gen_molDEMO_4_lig_gen_fit_add_uff.sdf', '/Users/song-inhyeok/Documents/BIChem/1V4S/gen_molDEMO_5_lig_gen_fit_add_uff.sdf', '/Users/song-inhyeok/Documents/BIChem/1V4S/gen_molDEMO_6_lig_gen_fit_add_uff.sdf', '/Users/song-inhyeok/Documents/BIChem/1V4S/gen_molDEMO_7_lig_gen_fit_add_uff.sdf', '/Users/song-inhyeok/Documents/BIChem/1V4S/gen_molDEMO_8_lig_gen_fit_add_uff.sdf', '/Users/song-inhyeok/Documents/BIChem/1V4S/gen_molDEMO_9_lig_gen_fit_add_uff.sdf', '/Users/song-inhyeok/Documents/BIChem/1V4S/gen_molDEMO_10_lig_gen_fit_add_uff.sdf', '/Users/song-inhyeok/Documents/BIChem/1V4S/gen_molDEMO_11_lig_gen_fit_add_uff.sdf', '

In [48]:
path = '/Users/song-inhyeok/Documents/BIChem/1V4S/gen_mol'
path2 = '/Users/song-inhyeok/Documents/BIChem/1V4S/liGAN_input.types'
    
file_list = [ 'DEMO_' + str(i) + '_lig_gen_fit_add_uff.sdf' for i in range(23)]
sample=file_list[0]
#print(file_list)

list_file = open(path2, 'r')
lines = list_file.readlines()

for i, file in enumerate(file_list):
    
    file_name = path + '/' + file
    obConversion = openbabel.OBConversion()
    obConversion.SetInAndOutFormats("sdf", "sdf")

    mol = openbabel.OBMol()
    obConversion.ReadFile(mol, file_name)
    print(mol)
    

<openbabel.openbabel.OBMol; proxy of <Swig Object of type 'OpenBabel::OBMol *' at 0x175e60360> >
<openbabel.openbabel.OBMol; proxy of <Swig Object of type 'OpenBabel::OBMol *' at 0x1769b3e40> >
<openbabel.openbabel.OBMol; proxy of <Swig Object of type 'OpenBabel::OBMol *' at 0x175e60180> >
<openbabel.openbabel.OBMol; proxy of <Swig Object of type 'OpenBabel::OBMol *' at 0x175e60c30> >
<openbabel.openbabel.OBMol; proxy of <Swig Object of type 'OpenBabel::OBMol *' at 0x17698c750> >
<openbabel.openbabel.OBMol; proxy of <Swig Object of type 'OpenBabel::OBMol *' at 0x17698c570> >
<openbabel.openbabel.OBMol; proxy of <Swig Object of type 'OpenBabel::OBMol *' at 0x17698c210> >
<openbabel.openbabel.OBMol; proxy of <Swig Object of type 'OpenBabel::OBMol *' at 0x17698cb10> >
<openbabel.openbabel.OBMol; proxy of <Swig Object of type 'OpenBabel::OBMol *' at 0x17698c3c0> >
<openbabel.openbabel.OBMol; proxy of <Swig Object of type 'OpenBabel::OBMol *' at 0x17698c870> >
<openbabel.openbabel.OBMol; pr

/opt/homebrew/Caskroom/miniforge/base/envs/office/lib/python3.8/site-packages/openbabel/__init__.py:14: UserWarning: "import openbabel" is deprecated, instead use "from openbabel import openbabel"
  warnings.warn('"import openbabel" is deprecated, instead use "from openbabel import openbabel"')
*** Open Babel Warning  in ReadMolecule
  Failed to kekulize aromatic bonds in MOL file (title is 0)

*** Open Babel Warning  in ReadMolecule
  Failed to kekulize aromatic bonds in MOL file (title is 0)

*** Open Babel Warning  in ReadMolecule
  Failed to kekulize aromatic bonds in MOL file (title is 0)

*** Open Babel Warning  in ReadMolecule
  Failed to kekulize aromatic bonds in MOL file (title is 0)

*** Open Babel Warning  in ReadMolecule
  Failed to kekulize aromatic bonds in MOL file (title is 0)

*** Open Babel Warning  in ReadMolecule
  Failed to kekulize aromatic bonds in MOL file (title is 0)

*** Open Babel Warning  in ReadMolecule
  Failed to kekulize aromatic bonds in MOL file (tit

In [51]:
obConversion = openbabel.OBConversion()
obConversion.SetInAndOutFormats("sdf", "sdf")

mol = openbabel.OBMol()

obConversion.ReadFile(mol,'/Users/song-inhyeok/Documents/BIChem/1V4S/gen_molDEMO_0_lig_gen_fit_add_uff.sdf')
for e in file_name:
    obConversion.ReadFile(mol,'/Users/song-inhyeok/Documents/BIChem/1V4S/gen_mol/DEMO_0_lig_gen_fit_add_uff.sdf')
    print(mol)

<openbabel.openbabel.OBMol; proxy of <Swig Object of type 'OpenBabel::OBMol *' at 0x176928810> >
<openbabel.openbabel.OBMol; proxy of <Swig Object of type 'OpenBabel::OBMol *' at 0x176928810> >
<openbabel.openbabel.OBMol; proxy of <Swig Object of type 'OpenBabel::OBMol *' at 0x176928810> >
<openbabel.openbabel.OBMol; proxy of <Swig Object of type 'OpenBabel::OBMol *' at 0x176928810> >
<openbabel.openbabel.OBMol; proxy of <Swig Object of type 'OpenBabel::OBMol *' at 0x176928810> >
<openbabel.openbabel.OBMol; proxy of <Swig Object of type 'OpenBabel::OBMol *' at 0x176928810> >
<openbabel.openbabel.OBMol; proxy of <Swig Object of type 'OpenBabel::OBMol *' at 0x176928810> >
<openbabel.openbabel.OBMol; proxy of <Swig Object of type 'OpenBabel::OBMol *' at 0x176928810> >
<openbabel.openbabel.OBMol; proxy of <Swig Object of type 'OpenBabel::OBMol *' at 0x176928810> >
<openbabel.openbabel.OBMol; proxy of <Swig Object of type 'OpenBabel::OBMol *' at 0x176928810> >
<openbabel.openbabel.OBMol; pr

*** Open Babel Error  in ReadFile
  Cannot read from /Users/song-inhyeok/Documents/BIChem/1V4S/gen_molDEMO_0_lig_gen_fit_add_uff.sdf


In [ ]:
path = '/Users/song-inhyeok/Documents/BIChem/1V4S/gen_mol'
path2 = '/Users/song-inhyeok/Documents/BIChem/1V4S/liGAN_input.types'
    
file_list = [ 'DEMO_' + str(i) + '_lig_gen_fit_add_uff.sdf' for i in range(23)]

#print(file_list)

list_file = open(path2, 'r')
lines = list_file.readlines()

for i, file in enumerate(file_list):
    file_name = path + '/' + file
    sdf = Chem.SDMolSupplier(file_name)
    
    
    mol_3d_list = open(file_name, 'r').read().split('$$$$')
    conf_id = lines[i].split('/')[-1].split('_')[-1].split('.')[0].replace('\n', '')
    
    for j, mol in enumerate(sdf):
        print(j, mol)
     